In [6]:
import os
import glob
import altair as alt
import pandas as pd
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [7]:
nfl_date = "20221018"


In [15]:
file_path = f"../data/team_ranks/_nfl_stat*{nfl_date}.csv"
file_paths = glob.glob(os.path.join(file_path))
pds = list()
for file_path in file_paths:
    df = pd.read_csv(file_path)
    pds.append(df)
df = pd.concat(pds)
def_stats = {
    "opponent-points-per-game":.6,
    "opponent-points-per-play":1,
    "opponent-1st-half-points-per-game":1.5,
    "opponent-2nd-half-points-per-game":1.6,
    "opponent-rushing-touchdowns-per-game":1.7,
    "opponent-passing-touchdowns-per-game":1.8,
    "opponent-yards-per-game":1.85,
    "opponent-yards-per-play":1.9,
    "opponent-rushing-yards-per-game":2,
    "opponent-yards-per-rush-attempt":2.1,
    "opponent-passing-yards-per-game":2.6,
    "opponent-yards-per-pass-attempt":3,
    "defensive-points-per-game":4,
    "sacks-per-game":10,
    "interceptions-per-game":11,
    "opponent-fumbles-per-game":12,
    }


stats = {
    "points-per-game":2,
    "points-per-play":3.5,
    "1st-half-points-per-game":3.9,
    "2nd-half-points-per-game":4,
    "rushing-touchdowns-per-game":4.1,
    "passing-touchdowns-per-game":4.2,
    "yards-per-game":4.5,
    "yards-per-play":4.8,
    "rushing-yards-per-game":4.9,
    "yards-per-rush-attempt":5,
    "rushing-play-pct":5.5,
    "passing-yards-per-game":5.6,
    "yards-per-pass-attempt":6,
    "passing-play-pct":7,
    "qb-sacked-per-game":10,
    "interceptions-thrown-per-game":11,
    "fumble-per-game":12,
    "penalties-per-game":13
    }

df["_stat"] = df.url.str.split("/", expand=True)[3]
df["Stat"] = df._stat.str.split("-").str.join(" ").str.title()
df["Stat"] = df.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")

In [16]:
df = df[df.Team.isin(["Chicago", "New England"])]


In [17]:
df

Unnamed: 0  Rank         Team  2022 Last 3 Last 1  Home  Away  2021  \
11          11    11  New England  23.5   30.3   38.0  27.5  21.5  26.6   
30          30    31      Chicago  15.5   13.7    7.0  16.3  14.7  18.3   
4            4     5  New England   4.7   16.3   23.0   9.0   2.5   7.2   
25          25    26      Chicago  -4.2   -6.7   -5.0   2.3 -10.7  -5.6   
13          13    14  New England  15.1   11.4   10.5  14.7  15.3  13.2   
..         ...   ...          ...   ...    ...    ...   ...   ...   ...   
22          22    23  New England  0.05   0.07   0.09  0.04  0.06  0.05   
7            7     8  New England   8.7   10.3    9.0  10.2   7.9   8.5   
20          20    21      Chicago   7.9    7.7    5.1   7.3   8.8   9.2   
7            7     8      Chicago  0.05   0.06   0.06  0.06  0.05  0.04   
30          30    31  New England  0.04   0.05   0.05  0.04  0.04  0.05   

          asof                                           url  \
11  2022-10-18                     /nfl/stat/points-per-game   
30  2022-10-18                     /nfl/stat/points-per-game   
4   2022-10-18              /nfl/stat/average-scoring-margin   
25  2022-10-18              /nfl/stat/average-scoring-margin   
13  2022-10-18                     /nfl/stat/yards-per-point   
..         ...                                           ...   
22  2022-10-18                  /nfl/stat/penalties-per-play   
7   2022-10-18  /nfl/stat/opponent-penalty-yards-per-penalty   
20  2022-10-18  /nfl/stat/opponent-penalty-yards-per-penalty   
7   2022-10-18         /nfl/stat/opponent-penalties-per-play   
30  2022-10-18         /nfl/stat/opponent-penalties-per-play   

                                 _stat                           Stat  
11                     points-per-game                      Points Pg  
30                     points-per-game                      Points Pg  
4               average-scoring-margin         Average Scoring Margin  
25              average-scoring-margin         Average Scoring Margin  
13                     yards-per-point                Yards Per Point  
..                                 ...                            ...  
22                  penalties-per-play             Penalties Per Play  
7   opponent-penalty-yards-per-penalty  Opp Penalty Yards Per Penalty  
20  opponent-penalty-yards-per-penalty  Opp Penalty Yards Per Penalty  
7          opponent-penalties-per-play         Opp Penalties Per Play  
30         opponent-penalties-per-play         Opp Penalties Per Play  

[452 rows x 13 columns]

In [13]:
chart_title = f"{nfl_date} Team Rankings"
y_cfgs = {"shorthand":"team_name", "title": ""}
x_cfgs = {"shorthand":"opp_wins:Q", "title":"Total 2021 Wins for 2022 Opponents"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
df = df_opp_wins

In [12]:
bars = alt.Chart(df, width=1000, height=1000, title=chart_title)\
.mark_bar(size=20, opacity=1)\
.encode(
    y=alt.Y(**y_cfgs, sort=alt.EncodingSortField(field="opp_wins", order="descending")),
    x=alt.X(**x_cfgs),
    color=alt.Color("Is Bears", scale=alt.Scale(domain=[True, False], range=[secondary_color, "white"]), legend=None),
    tooltip=[alt.Tooltip("team_name", title="Team Name")]
)

text = bars.mark_text(
    dx=15,
    fontSize=17,
).encode(
    text='opp_wins'
)

(bars + text).resolve_scale(y="independent")\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)\
.configure_axisRight(labelColor=primary_color, titleColor=primary_color, labelFontSize=0, titleFontSize=0)\
.configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)\
.configure_title(color=secondary_color, fontSize=25)
#.configure_legend(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10,cornerRadius=10,orient='bottom-right', labelFontSize=15, titleFontSize=15

alt.LayerChart(...)